In [12]:
!pip install gdown --quiet

# Replace this URL with your public shareable link from Google Drive
url = "https://drive.google.com/file/d/12UPNORSnMAi_gnyODkG3wEcuPTON3WBp"

!gdown {url} -O best_model.pth


/usr/local/lib/python3.12/dist-packages/gdown/parse_url.py:48: UserWarning: You specified a Google Drive link that is not the correct link to download a file. You might want to try `--fuzzy` option or the following url: https://drive.google.com/uc?id=None
  warnings.warn(
Downloading...
From: https://drive.google.com/file/d/12UPNORSnMAi_gnyODkG3wEcuPTON3WBp
To: /content/best_model.pth
8.91kB [00:00, 15.0MB/s]


In [13]:
import torch
import segmentation_models_pytorch as smp

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

# 1️⃣ Create the model structure exactly like before
model = smp.UnetPlusPlus(
    encoder_name="efficientnet-b4",
    encoder_weights=None,  # None because we will load our trained weights
    in_channels=3,
    classes=1,
    decoder_dropout=0.2
).to(device)

# 2️⃣ Load the saved checkpoint safely
# PyTorch 2.6+ defaults to weights_only=True; set it to False for full checkpoint
checkpoint_path = "best_model.pth"
model.load_state_dict(torch.load(checkpoint_path, map_location=device, weights_only=False))

# 3️⃣ Set model to evaluation mode
model.eval()

print("Model loaded successfully!")


Using device: cpu


UnpicklingError: invalid load key, '<'.

In [ ]:
# Example for a single image
import cv2
import numpy as np
from torchvision import transforms

img_path = "test_images/example.png"
img = cv2.imread(img_path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img = cv2.resize(img, (960, 540)) / 255.0
img = transforms.ToTensor()(img).unsqueeze(0).to(device)

with torch.no_grad():
    pred = torch.sigmoid(model(img))
    pred_mask = (pred > 0.5).float().cpu().squeeze().numpy()
